In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from collections import defaultdict
#import ipynb.fs.full.config as cfg
import pathlib
import pandas as pd
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
from email.utils import parsedate_tz, mktime_tz
import datetime
import numpy as np
import plotly.express as px
import kaleido

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


class snitchan_analytics(object):
    def __init__(self):
    #Shows basic usage of the Gmail API.
    #Connect to gmail using Oauth.
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('client_secret_gmail.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        self.service = build('gmail', 'v1', credentials=creds)
    #Function that will connect all the analysis functions together
    def analysis(self, search_term, gmail_results):
        DfDate = defaultdict(list)
        self.process_gmail(gmail_results, DfDate)
        
    def process_gmail(self, gmail_results, DfDate):
        for msg in gmail_results:
            txt = self.service.users().messages().get(userId='me', id=msg['id']).execute()
            payload = txt['payload']
            headers = payload['headers']

            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                if d['name'] == 'Date':
                    date = d['value']
                    
            #Store date in data frame
            date = mktime_tz(parsedate_tz(date))
            DfDate['Date'].append(date)
            
            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)

            # Now, the data obtained is in lxml. So, we will parse 
            # it with BeautifulSoup library
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()

            # Printing the subject, sender's email and message
            #print("Subject: ", subject)
            #print("From: ", sender)
            #print("Message: ", body)
            #print("Date: ", date)
            #print('\n')
        DfDate = pd.DataFrame(DfDate)
        DfDate['Date'] = pd.to_datetime(DfDate['Date'], unit='s')
        
        DfDate = DfDate.groupby(DfDate['Date'].dt.date).size().reset_index(name='Count')
        
        fig = px.bar(DfDate, x='Date', y="Count")
        fig.write_image('TimeSeries.png')
        
        f = open("analytics results.html", "a")
        f.write("<h3>TimeSeries:</h3>\n")
        #if not messages:
            #f.write('No Results found.'+ '\n')
        #else:
            
        f.write("<ol>\n")
        f.write("<img src='TimeSeries.png'/>")
        f.write("</ol>\n")       
        f.close()
        
        
        #DfDate['Date'] = pd.to_datetime(DfDate['Date'], format='%a, %d %b %Y %H:%M:%S %z (%Z)')
        print(DfDate)
